## 1 Introduction

This is the notebook for the Gatys model from [the Gatys et al. paper](https://doi.org/10.1109/CVPR.2016.265). This serves as the main stylisation baseline of the NST project. This project will define a 
gradient descent process for an optimization loop.

## 1.1 Checking the environment of the project

Check the Python version.

In [ ]:
!python --version

Checking if the project is in Kaggle.

In [ ]:
import sys
KAGGLE_DIR_PATH = "/kaggle/input/"
KAGGLE_SCRIPT_PATHS =  ["/kaggle/input/gatys_model/gatys_model", "/kaggle/input/shared-utils/shared-utils", "/kaggle/input/video_utils/video_utils"]
on_kaggle : bool = True if any("kaggle" in path for path in sys.path) else False

Checking if the project is in Google Colab.

In [1]:
import importlib
def check_if_on_colab():
    on_colab = 'google.colab' in str(get_ipython())
    return on_colab

In [2]:
on_colab = check_if_on_colab()

Adding base paths for the project to allow the import to work the same way in colab and Kaggle as it would in a local environment.

In [ ]:
import os
def add_base_script_paths():
  if on_colab:
    sys.path.append('/content/drive/MyDrive')
    nb_path = '/content/drive/MyDrive/Library'
    os.makedirs(nb_path, exist_ok=True)
    sys.path.insert(0,nb_path)
    
  elif on_kaggle:
    kaggle_dir_path = KAGGLE_DIR_PATH
    kaggle_script_paths = KAGGLE_SCRIPT_PATHS
    sys.path.insert(1, kaggle_dir_path)
    for x in  kaggle_script_paths:
      if x not in sys.path:
        sys.path.insert(1, x)
  else:
    sys.path.append('..')

In [ ]:
add_base_script_paths()

Create a function to see if Google Drive is mounted.

In [ ]:
def is_drive_mounted():
    drive_path = "/content/drive"
    return os.path.exists(drive_path)

In [ ]:
drive_available = is_drive_mounted()

## 2 Image-based Style Transfer

Set the avalible cuda visible devices to 0, to prevent from using two GPUs at once.

In [ ]:
import os
# ensures it can only access the first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Control the log output of the tqdm class.

In [ ]:
TQDM_DISABLE=1

Import the tensorflow library

In [ ]:
import tensorflow as tf

Prints the tensorflow version used by the project

In [ ]:
# prints the tensorflow version
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


Download the shared util and helper functions library for usage in the project.

In [ ]:
def clone_repo(should_clone = False,token = ""):
    if should_clone:
        !git config --global url."https://{token}@github.com/".insteadOf "https://github.com/"
        !git clone --filter=blob:none --no-checkout https://github.com/Averagenormaljoe/Neural-Style-Transfer.git
        !cd Neural-Style-Transfer
        !git sparse-checkout init --cone
        !git sparse-checkout set shared_utils
        !git sparse-checkout set video_utils
        !git sparse-checkout set helper_functions
        !git sparse-checkout set gatys_functions
        !git checkout

In [ ]:
from shared_utils.colab_functions import download_libraries
if on_colab:
    nb_path = '/content/drive/MyDrive/Library'
    download_libraries(nb_path)

List the GPUs use by the project

In [ ]:
from helper_functions.list_devices import find_all_gpus

SyntaxError: invalid syntax (2059008399.py, line 1)

Get the GPU avaliable in the project

In [ ]:
find_all_gpus()

Get the available CPUs in the project.

In [ ]:
from helper_functions.list_devices import show_cpu

Available CPUs: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


Show the number of CPUs.

In [ ]:
show_cpu()

Set the project to use the CPU and GPU of interest.

In [6]:
GPU_in_use: int = 0
CPU_in_use: int = 0

In [ ]:
device_config = {"gpu" : GPU_in_use, "cpu" : CPU_in_use}

In [7]:
from helper_functions.device_helper import get_device

Define a function to see if there are any gpu available

In [ ]:
from helper_functions.list_devices import show_gpu

No GPU found


Next call the function to show the currently used GPUs.

In [ ]:
show_gpu(GPU_in_use)

Let define the paths for the demo images of the project

In [ ]:
import keras
base_image_path = "../demo_images/san.png"
style_reference_image_paths = ["../demo_images/starry_night.png"]
style_reference_path = style_reference_image_paths[0]

Define a function to get the size of the image

In [ ]:
from helper_functions.image_loader import get_size

Retrieve the size of the image that the project plans to load.

In [ ]:
img_width, img_height = get_size(base_image_path)

In [ ]:
from helper_functions.helper import  preprocess_image, deprocess_image
from shared_utils.loss_functions import style_loss, content_loss, total_variation_loss

Define the single content loss for each layer.

In [ ]:
total_variation_weight = 1e-6
single_style_weight = 1e-6
single_content_weight = 2.5e-8

In [ ]:
from shared_utils.gatys_network import get_content_layer_names,get_style_layer_names,get_style_weights, get_content_weights

The project will specify the loss network used for the gradient descent loop.

In [ ]:
# the chosen loss network
chosen_loss_network : str = "vgg19"

The next code snippet defines custom options for layers and weight, which is useful if we are using a loss network that is not 'vgg-19'.  
If the 'use_custom' option is false it call the 'get_default_NST_layers' function as the return parameter.

In [ ]:
from gatys_functions.get_layers import get_layers

Run the function to get the default layers.

In [ ]:
results = get_layers(False,chosen_loss_network)
style_layer_names, content_layer_names, style_weights, content_weights = results

In [ ]:
config_layers = {
    "style" : style_layer_names,
    "content" : content_layer_names
}

Next, the project will define a function to output the VGG-19 loss network.

In [ ]:
from gatys_functions.get_model import get_model

Now, the project will retrieve the VGG-19 network.

In [ ]:
feature_extractor = get_model(chosen_loss_network,img_width,img_height, config_layers=config_layers)

Let's define a function to compute the losses for the model. This function will iterate through each of the layers, calculating the features of the image and returning the content and style loss for the image.  
This also calculates the total variation loss, along with additional losses (psnr, ssim, art, etc.) specified by the application. By default, it will only compute content, style, and total variation loss.

In [ ]:
from gatys_functions.compute_loss import compute_loss

In [ ]:
from tensorflow.keras import layers

Sets whether to use the float 16 policy.

In [ ]:
from shared_utils.policy import control_policy

Call the policy function.

In [ ]:
control_policy(enable_mixed_precision=False)

Create the compute loss and grads function. This will define a gradient tape, which gradients will be used to update the  
stylized image (combination image) between each iteration. If multi style images are provided for the loss function, it will  
compute them as a part of multi-neural style transfer.

In [ ]:
from gatys_functions.compute_loss_and_grads import compute_loss_and_grads

Set the function for processing multi style images for multi neural style transfer.

Create a noise function to supply initial noise for the content image.

In [ ]:
from helper_functions.image_helper import add_noise_to_image

In [ ]:
from helper_functions.helper import match_style_color_to_base

Define a function to easily preprocess the base, style and combination image.

In [ ]:
from gatys_functions.preprocess_NST_images import preprocess_NST_images

In [ ]:
import math
import os

Define a function to call the 'compute_loss_and_grads' function to apply the gradients to the combination image.

In [ ]:
from video_utils.mask import warp_previous_frames

In [ ]:
from video_utils.helper.get_flow_and_wrap import get_flow_and_wrap
from video_utils.helper.reset_warp_frames import reset_warp_frames

In [ ]:
from gatys_functions.apply_style_transfer_step import apply_style_transfer_step

In [ ]:
from helper_functions.bestImage import BestImage

Now, the project will define the training loop. This loop represent the gradient descent process, and will accept the path of a content and style images before
generating the combination image.

In [ ]:
from helper_functions.log_funcs import create_log_dir

This will handle the config for the training loop.

In [ ]:
from helper_functions.ConfigManager import ConfigManager

Next, let define the loop manager for handling the steps in the process.

In [ ]:
from shared_utils.compute_custom_losses import CustomLosses

Next, the project will define the training loop.

In [ ]:
from gatys_functions.LoopManager import LoopManager

Define the folders for the content and style images.

In [ ]:
base_dir = "user_study_images/"
content_folder = f"{base_dir}content"
style_folder = f"{base_dir}style"

Define a function to collect the image files from the directory with the specified file extension.

In [ ]:
from shared_utils.file_nav import get_image_files

Next, use the function to retrieve the content and style files.

In [10]:
content_images = get_image_files(content_folder)
style_images = get_image_files(style_folder)

In [ ]:
content_images

In [ ]:
style_images

Define a function that allows the project to easily update the style model and content layers, along with weights.

Setup the hyperparameter configuration for the project.

In [ ]:
config = {
    "optimizer": "adam",
    "ln": "vgg19",
    "lr": 1.0,
    "size": (img_width,img_height),
    "content_layer_names": content_layer_names,
    "style_layer_names": style_layer_names,
    "feature" : feature_extractor,
    "c_weight": single_content_weight,
    "s_weight": single_style_weight,
    "tv_weight": total_variation_weight,
    "video_mode": False
}

Setup the loop manager class.

In [ ]:
loop_manager = LoopManager(config)

Next, define two function that will use the training loop but track the results of each image in the list and store into an array.

In [ ]:
from shared_utils.file_nav import get_base_name

In [ ]:
# iterate through the content and style images
def loop_through_images(content_images, style_images,loop_manager : LoopManager,config = {}):
    image_set = []
    best_image_set = []
    image_data_logs = []
    image_paths = []
    for content_path in content_images:
        content_name = get_base_name(content_path)
        for style_path in style_images:
            style_name = get_base_name(style_path)
            results = loop_manager.training_loop(
                content_path, style_path,
                content_name, style_name, config=config
            )
            if results is None:
                print(f"Failed loop for ({content_name}) and ({style_name}). Next loop...")
                continue
            generated_images, best_image,log_data = results
            image_set.append(generated_images)
            best_image_set.append(best_image)
            image_data_logs.append(log_data)
            image_paths.append((content_path, style_path))
    return image_set, best_image_set, image_data_logs, image_paths

c:\Users\Layo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(3, 400, 535, 3))
  warnings.warn(msg)


TypeError: in user code:

    File "C:\Users\Layo\AppData\Local\Temp\ipykernel_19984\189228123.py", line 7, in compute_loss_and_grads  *
        loss = compute_loss(
    File "C:\Users\Layo\AppData\Local\Temp\ipykernel_19984\2033682352.py", line 6, in compute_loss  *
        layer_features = features[content_layer_name]

    TypeError: unhashable type: 'list'


Execute the function on the content and style directory paths in the project.

In [ ]:
image_set, best_image_set, image_data_logs, image_paths = loop_through_images(content_images, style_images,loop_manager,config)

The next function collects the image information from the list in the loop to display the data in matplotlib.

In [ ]:
def get_image_info(i):
    generated_images = image_set[i]
    best_image = best_image_set[i]
    iterations = image_data_logs[i]["iterations"]
    losses = image_data_logs[i]["loss"]
    image_path = image_paths[i]
    return generated_images, best_image, iterations, losses, image_path

Get the image data using the function.

In [ ]:
generated_images,best_image, iterations, losses, image_path = get_image_info(0)

In [ ]:
from helper_functions.display_results import display_NST_results

Displays the results as a matplotlib graph.

In [ ]:
generated_images,best_image, iterations, losses, image_path = get_image_info(0)
display_NST_results(generated_images, best_image, iterations, losses, image_path, start_index = 0, config = config)

In [ ]:
def save_NST_results(start_ranges = [0]):
    length = len(image_set)
    for i in range(length):
        for start_i in start_ranges:
                generated_images,best_image, iterations, losses, get_image_paths = get_image_info(i)
                display_NST_results(generated_images, best_image, iterations, losses, get_image_paths, start_index = start_i)
            

In [ ]:
start_ranges = [0]
save_NST_results(start_ranges)

Place the results into a table and convert them to a '.csv' table to store the hardware metrics.

In [ ]:
from helper_functions.table_saver import  save_and_show_all
save_and_show_all(image_data_logs, image_paths)

## 3 Video Style Transfer 

This section will define a set of functions to stylize a video using a sample url and images before storing the video into a dictionary.

In [ ]:
from video_utils.helper.loop_through_videos import loop_through_videos

Let define the style image paths and the video path to loop through.

In [ ]:
style_paths = ["user_study_images/style/starry_night.png","user_study_images/style/picasso.jpg", "user_study_images/style/art16.jpg", "user_study_images/style/art12.jpg", "user_study_images/style/art7.jpg"]
video_content_path = "demo_video/man_at_sea_sliced.mp4"
apply_video = loop_manager.training_loop
total_logs = loop_through_videos(apply_video,style_paths, video_content_path,config=config)

In [ ]:
from video_utils.save_video_logs_table import save_video_logs_table

End the notebook at this point.

In [ ]:
save_video_logs_table(total_logs, "videos")

In [ ]:
top_folder_name = "Gatys_model"

In [ ]:
video_save_path : str = f"{top_folder_name}"

In [ ]:
import time
def run_loop(tw = 1.0, save_path : str) -> list[dict]:
    luminance_versions = [1,2,3,0]
    style_paths = style_images
    video_content_path = "demo_video/man_at_sea_sliced.mp4"
    time_logs = []
    for v in luminance_versions:
        long_term = [True,False]
        flow_conditions = [True]
        mask_conditions = [True,False]
        is_luminance = False if v == 0 else True
        for term in long_term:
            for is_flow in flow_conditions:
                for is_mask in mask_conditions:
                    config = {
                                
                            "is_mask" : is_mask,
                            "is_multi_pass" : is_mask,
                            "is_flow" : is_flow,
                            "verbose" : 1,
                            "frames_limit" : 10000,
                            "long_term" : term,
                            "is_luminance" : is_luminance,
                            "temporal_weight" : tw,
                            "luminance_version" : v,
                    
                    }
                    log = config.copy()
                    start_time = time.time()
                    cpu_start_time =  time.process_time()
                    v_cond = "off" if not is_luminance else str(v)
                    mask_cond = "off" if not is_mask else str(is_mask)
                    flow_cond = "off" if not is_mask else str(is_flow)
                    video_save_path = f"{top_folder_name}_johnson_multi_pass_{mask_cond}_luminance_{v_cond}_long_term_{term}_tw_{tw}_flow_{flow_cond}_mask_{mask_cond}"
                    total_logs = loop_through_videos(apply_video,style_paths, video_content_path,f"{video_save_path}",config=config)
                    save_video_logs_table(total_logs,save_path,prefix= f"{video_save_path}_man_logs")
                    end_time = (time.time() - start_time)
                    cpu_end_time = (time.process_time() - cpu_start_time)
                    log.update({"time": end_time, "cpu_time" : cpu_end_time, "path": video_save_path})
                    time_logs.append(log)
    return time_logs




In [ ]:
from shared_utils.helper import create_dir


save_path = "video_data"
create_dir(save_path=save_path)
time_logs = run_loop()

In [ ]:
import pandas as pd

df_time = pd.DataFrame(time_logs)
df_time_csv_path = os.path.join(save_path, f"{top_folder_name}_video_time.csv")
df_time.to_csv(df_time_csv_path, index=False)

End of the notebook.